In [1]:
import sys; sys.path.append("../")

import pandas as pd

from lib.match import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)
from lib.date import combine_date_columns
from clean.new_orleans_harbor_pd_pprr import clean_personnel_2008, clean_personnel_2020
from match.new_orleans_harbor_pd import prepare_post_data, match_uid_with_cprr
from lib.columns import rearrange_personnel_columns, rearrange_personnel_history_columns

In [2]:
df20 = clean_personnel_2020()
df20

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../lib/clean.py:156: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\d\.]", "").astype("float64")


,first_name,last_name,rank_desc,hourly_salary,middle_initial,effective_date,agency,data_production_year,uid,perhist_uid,...,resign_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,rank_year,rank_month,rank_day
0,kendall,turner,police officer 1-a,10.10,r,1/3/2007,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,c3dc1f7275ff3681ca673813d786b798,...,,2007,1,3,2007,1,3,2007,1,3
1,kendall,turner,police officer 2-a,15.18,r,1/4/2008,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,13bf83f9ed69849fc2ad61e5329adca5,...,,2007,1,3,2008,1,4,2008,1,4
2,kendall,turner,police officer 2-a,17.08,r,10/1/2012,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,043c9de3e938d8cd2525def644ad3c64,...,,2007,1,3,2012,10,1,,,
3,kendall,turner,police officer 2-a,17.76,r,10/1/2013,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,59fccb5c71605c1916f8939aa6aa1381,...,,2007,1,3,2013,10,1,,,
4,kendall,turner,police officer 2-a,18.47,r,10/1/2014,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,d3e867074632239a401ae0f9b7bcb0b5,...,,2007,1,3,2014,10,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,linda,st cyr,police officer 3-a,19.50,l,10/1/2007,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,2c1c0b941a8c74083a15df59e0c1cbc7,...,31,1982,12,27,2007,10,1,,,
898,linda,st cyr,police officer 3-a,22.31,l,10/1/2008,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,8162e05dc991530289bc4115e449d25a,...,31,1982,12,27,2008,10,1,,,
899,linda,st cyr,police officer 3-a,21.45,l,11/1/2007,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,f31cde8d053c5701bc9db56facc2fa21,...,31,1982,12,27,2007,11,1,,,
900,linda,st cyr,police officer 3-a,18.75,l,7/1/2007,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,eea3e1b2bf424806c2a2ec9c4c363fb1,...,31,1982,12,27,2007,7,1,,,


In [3]:
df08 = clean_personnel_2008()
df08

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../lib/clean.py:156: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\d\.]", "").astype("float64")


,first_name,middle_initial,last_name,rank_desc,effective_date,hourly_salary,agency,data_production_year,uid,perhist_uid,...,hire_day,resign_year,resign_month,resign_day,pay_effective_year,pay_effective_month,pay_effective_day,rank_year,rank_month,rank_day
0,cheryl,e,st pe,police corporal--port,2/28/1992,9.99,New Orleans Harbor PD,2008,000526fa05739e97b61343513a92dbc7,598ce57575a1bf7974fb8ebd969d5c4e,...,19,1992,2,28,1992,2,28,1992,2,28
1,larry,m,francingues,police officer 2--port,1/19/2002,11.32,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,62476dc1ad5918dbed71d25ef147a9a7,...,19,2005,12,19,2002,1,19,2002,1,19
2,larry,m,francingues,police officer 2--port,1/19/2003,11.77,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,8ac42cec937710ab76d6b47f993d8588,...,19,2005,12,19,2003,1,19,,,
3,larry,m,francingues,police officer 2--port,1/19/2004,12.24,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,ca41c00d4ecfe5a375a6d4a1205c7a80,...,19,2005,12,19,2004,1,19,,,
4,larry,m,francingues,police officer 2-a,1/19/2005,12.73,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,2d6b84d1da2fdaa61df7319d86941cea,...,19,2005,12,19,2005,1,19,2005,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,antoine,a,derouen jr,police officer 2--port,3/8/2003,12.75,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,e60fefbd16588ed32feee633ef7b0d07,...,8,2006,12,11,2003,3,8,,,
163,antoine,a,derouen jr,police officer 2--port,3/8/2004,13.26,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,ea0656b90251a0f6ffdb62c4200e43b5,...,8,2006,12,11,2004,3,8,,,
164,antoine,a,derouen jr,police officer 2--port,3/8/2005,13.79,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,d8f6e28091455761ce48a745212c2346,...,8,2006,12,11,2005,3,8,,,
165,antoine,a,derouen jr,police officer 2-a,3/8/2006,14.34,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,558f7b9edb26f64d9d61d4fe9e7f26c6,...,8,2006,12,11,2006,3,8,2006,3,8


In [5]:
cprr = pd.read_csv("../data/clean/cprr_new_orleans_harbor_pd_2020.csv")
matcher = match_uid_with_cprr(cprr, df20)

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [6]:
matcher.get_sample_pairs()

AttributeError: 'DataFrame' object has no attribute 'get_sample_pairs'